In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from darts import TimeSeries

from darts.metrics import mape, mse, rmse, mae
from darts.models.filtering.moving_average_filter import MovingAverageFilter

from utils import print_metrics, plot_forecast, backtest, Dataset, TimeseriesExperiment

import pickle

import os

In [3]:
def get_series_from_dataframe(dataframe, column_name):
  # consts 
  TRAIN_DAYS = 60
  PREDICTION_HOURS = 24

  s = TimeSeries.from_dataframe(dataframe, time_col='ds', value_cols=column_name, freq=None, fill_missing_dates=True)
  s = s.slice_n_points_before(s.end_time(), TRAIN_DAYS*24*60)
  s = s.resample('10min', method='pad')

  ma_filter = MovingAverageFilter(12)
  s = ma_filter.filter(s)

  return s

def get_series_from_dataframe_xgboost(df: pd.DataFrame) -> TimeSeries:
  df['Global_active_power'] = pd.to_numeric(df['Global_active_power'], errors='coerce').astype(np.float32)
  df['Global_reactive_power'] = pd.to_numeric(df['Global_reactive_power'], errors='coerce').astype(np.float32)
  df['Voltage'] = pd.to_numeric(df['Voltage'], errors='coerce').astype(np.float32)
  df['Global_intensity'] = pd.to_numeric(df['Global_intensity'], errors='coerce').astype(np.float32)
  df['Sub_metering_1'] = pd.to_numeric(df['Sub_metering_1'], errors='coerce').astype(np.float32)
  df['Sub_metering_2'] = pd.to_numeric(df['Sub_metering_2'], errors='coerce').astype(np.float32)
  df['Sub_metering_3'] = pd.to_numeric(df['Sub_metering_3'], errors='coerce').astype(np.float32)

  # fill NA values with the previous value
  df = df.ffill()

  return get_series_from_dataframe(df, 'Global_active_power')

def load_electricity_data(): 
  df = pd.read_csv('data/household_power_consumption.txt', sep=';', parse_dates={'ds': ['Date', 'Time']})

  return get_series_from_dataframe_xgboost(df)


In [4]:
XGBoost_dataset = Dataset(load_electricity_data(), 'electricity')

/var/folders/fq/7hdv8yqd3ld281d0kdv2ylc80000gn/T/ipykernel_58395/1012107434.py:30: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/household_power_consumption.txt', sep=';', parse_dates={'ds': ['Date', 'Time']})
/var/folders/fq/7hdv8yqd3ld281d0kdv2ylc80000gn/T/ipykernel_58395/1012107434.py:30: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('data/household_power_consumption.txt', sep=';', parse_dates={'ds': ['Date', 'Time']})


In [5]:
from darts.models.forecasting.xgboost import XGBModel

xgb_model = XGBModel(
  lags=24 * 7 * 6,
  output_chunk_length=100,
)

xgb_experiment = TimeseriesExperiment(xgb_model, XGBoost_dataset, 'electricity', { 'lags': 24 * 7 * 6, 'output_chunk_length': 100, 'forecast_horizon': 24 })

In [6]:
xgb_experiment.run()

ValueError: Please pass exactly one of the arguments 'forecast_horizon', 'val_target_series' or 'use_fitted_values'.


URWANO electricity


ValueError: Please pass exactly one of the arguments 'forecast_horizon', 'val_target_series' or 'use_fitted_values'.